<a href="https://colab.research.google.com/github/ferdinandrafols/IA_LLMs/blob/main/gsi073_aula0_luong_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

Esta tarefa é inverter sequências de caracteres. Exemplo: **aabcd** em **dcbaa**.


In [ ]:
import torch
import torch.nn as nn
import random
import torch.nn.functional as F

chars = list("abcd ")
vocab = {ch: i for i, ch in enumerate(chars)} # Cada letra, ganha um número
inv_vocab = {i: ch for ch, i in vocab.items()}# Tabela de decodificação
vocab_size = len(vocab)

def encode(s): # Codifica letras em números
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t): # Decodifica números em letras
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5): # Cria novas sequências
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# Gerar dados
pairs = [(encode(s), encode(s[::-1])) for s in [random_seq() for _ in range(50000)]]

max_len = max(len(x) for x, _ in pairs) # pega maior sequência

def pad(x):  # Preenche conjunto de dados em pad no último índice
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Calculando a acurácia

In [ ]:
correct_predictions = 0
total_predictions = 0

# Using a subset of the data for testing to avoid long computation
# Let's use 1000 examples from the 'pairs' dataset
num_test_examples = 1000

for i in range(num_test_examples):
    input_tensor, target_tensor = pairs[i]

    # Decode the input and target tensors to strings
    s = decode(input_tensor)
    actual_inverted_s = decode(target_tensor)

    # Get the model's prediction
    # The predict function pads with spaces, so we need to clean it up for comparison
    predicted_inverted_s_padded = predict(model, s, max_len=len(s))
    predicted_inverted_s = predicted_inverted_s_padded.replace(' ', '') # Remove padding spaces

    # Compare and count
    if predicted_inverted_s == actual_inverted_s:
        correct_predictions += 1
    total_predictions += 1

accuracy = (correct_predictions / total_predictions) * 100
print(f"Acurácia do modelo em {num_test_examples} exemplos: {accuracy:.2f}%")

## Veja um par

In [ ]:
print(pairs[1])

# Definição do modelo Seq2Seq com GRU

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        outputs, h = self.gru(x)
        return outputs, h   # <--- ESSENCIAL

In [ ]:
class LuongAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, decoder_hidden, encoder_outputs):
        """
        decoder_hidden: (B, 1, H)
        encoder_outputs: (B, S, H)

        Retorna:
          context: (B, 1, H)
          attn_weights: (B, 1, S)
        """

        # score = h_t · h_s^T
        # (B, 1, H) x (B, H, S) -> (B, 1, S)
        attn_scores = torch.bmm(decoder_hidden, encoder_outputs.transpose(1, 2))

        attn_weights = F.softmax(attn_scores, dim=-1)  # normaliza nos steps da source

        # context = soma ponderada
        # (B, 1, S) x (B, S, H) -> (B, 1, H)
        context = torch.bmm(attn_weights, encoder_outputs)

        return context, attn_weights

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.attn = LuongAttention()

        # Luong concat: concatena hidden + context
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, x, h, encoder_outputs):
        """
        x: tokens anteriores corretos  (B, T)
        h: estado inicial do decoder   (1, B, H)
        encoder_outputs: todos os h_s  (B, S, H)
        """
        x = self.embed(x)  # (B, T, E)

        outputs = []
        seq_len = x.size(1)
        hidden = h

        for t in range(seq_len):
            inp = x[:, t:t+1]  # (B, 1, E)

            out_t, hidden = self.gru(inp, hidden)   # out_t: (B,1,H)

            # Atenção
            context, attn_w = self.attn(out_t, encoder_outputs)

            # concatenação [out_t ; context]
            combined = torch.cat([out_t, context], dim=-1)

            logits = self.fc(combined)  # (B,1,V)
            outputs.append(logits)

        outputs = torch.cat(outputs, dim=1)  # (B, T, V)
        return outputs, hidden


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        encoder_outputs, h = self.encoder(src)
        logits, _ = self.decoder(tgt[:, :-1], h, encoder_outputs)
        return logits

# Código para usar o modelo treinado: inferência

In [ ]:
def decode_step(decoder, token, h, encoder_outputs):
    """
    Executa um passo de decodificação:
    - token: tensor (B,1)
    - h: estado oculto do decoder (1,B,H)
    - encoder_outputs: (B,S,H)
    """
    logits, h = decoder(token, h, encoder_outputs)  # (B,1,V)
    next_token = logits[:, -1, :].argmax(-1, keepdim=True)  # (B,1)
    return next_token, h


def predict(model, seq, max_len=10):
    model.eval()
    with torch.no_grad():
        # codifica entrada
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)

        # encoder agora retorna (encoder_outputs, h)
        encoder_outputs, h = model.encoder(src)

        # token inicial (ex: espaço ou <sos>)
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)

        seq_invertida = []
        for _ in range(max_len):
            token, h = decode_step(model.decoder, token, h, encoder_outputs)
            seq_invertida.append(token.item())

        return decode(seq_invertida)


# Preparação para treino

In [ ]:
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' ']) # ignora o pad: " "
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

# Execução do treino

In [ ]:
for epoch in range(10):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt.zero_grad()
        logits = model(xb, yb)
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

# Vamos testar

In [ ]:
count = 0
for _ in range(100):
    s = random_seq()
    pred = predict(model, s, max_len=len(s))
    print(f"{s} -> {pred}")
    if s[::-1] == pred:
        count += 1
print(f"Acertos: {count}")


## Gerar Matriz de Scores de Atenção

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def _decode_step_with_attention(decoder, token, h, encoder_outputs):
    """
    Executa um passo de decodificação e retorna o próximo token, estado oculto e pesos de atenção.
    """
    # Embed o token atual
    inp = decoder.embed(token)  # (B, 1, E)

    # Passa pelo GRU do decoder
    out_t, h = decoder.gru(inp, h)   # out_t: (B,1,H)

    # Calcula a atenção
    context, attn_w = decoder.attn(out_t, encoder_outputs) # attn_w é (B, 1, S)

    # Concatena out_t e context e passa pela camada linear para obter os logits
    combined = torch.cat([out_t, context], dim=-1)
    logits = decoder.fc(combined)  # (B,1,V)

    # Seleciona o próximo token com a maior probabilidade
    next_token = logits[:, -1, :].argmax(-1, keepdim=True)  # (B,1)

    return next_token, h, attn_w

def get_attention_matrix(model, seq):
    model.eval()
    with torch.no_grad():
        # Codifica a sequência de entrada e adiciona padding
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)

        # Passa pelo encoder para obter as saídas e o estado oculto inicial do decoder
        encoder_outputs, h = model.encoder(src)

        # Token inicial (espaço)
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)

        attention_weights_list = []
        decoded_tokens = []

        # Decodifica até o comprimento máximo para capturar a atenção em cada passo
        for _ in range(max_len):
            next_token, h, attn_w = _decode_step_with_attention(model.decoder, token, h, encoder_outputs)
            attention_weights_list.append(attn_w.squeeze(0).cpu().numpy()) # (1, S) -> (S,) e anexa
            decoded_tokens.append(next_token.item())
            token = next_token

        # Empilha todos os pesos de atenção para formar a matriz (target_len, source_len)
        attention_matrix = np.vstack(attention_weights_list)

        # Prepara os rótulos para o heatmap
        padded_src_str = decode(src.squeeze(0))
        input_labels = list(padded_src_str)
        output_labels = list(decode(decoded_tokens))

        return attention_matrix, input_labels, output_labels

### Visualização da Matriz de Atenção

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Exemplo de uso com uma sequência arbitrária
input_sequence = random_seq() # Ou escolha uma sequência específica como "baca"

attn_matrix, input_labels, output_labels = get_attention_matrix(model, input_sequence)

plt.figure(figsize=(10, 8))
sns.heatmap(attn_matrix, cmap="viridis", cbar=True,
            xticklabels=input_labels, yticklabels=output_labels,
            linewidths=.5, linecolor='lightgray',
            annot=True, fmt=".2f") # Adicionado annot=True e fmt para formatar os valores
plt.xlabel("Sequência de Entrada (com padding)")
plt.ylabel("Sequência de Saída (com padding)")
plt.title(f"Matriz de Scores de Atenção para \"{input_sequence}\"")
plt.tight_layout()
plt.show()

## Contagem de parâmetros do modelo

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total de parâmetros no modelo: {total_params}")

# Exercício
Compare o resultado do uso do encoder-decoder com atenção com o encoder-decoder sem atenção.